In [1]:
import nibabel as nb
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from os.path import join
from glob import glob as glob
import pathos.multiprocessing as multiprocessing
from functools import partial
import nighres
import csv
import subprocess
#%pylab

# Coreg cortex

In [3]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
ls_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Coreg/r_coreg_surf_all/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'

all_dirs = glob(join(source_dir,'*'))
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [4]:
#I
files = "cortex_ls_r"

rows=[]
for sub_id in sub_ids:
    ls = nb.load(join(ls_dir,sub_id,files+".nii.gz"))
    ls_d = ls.get_data()
    ls_d[ls_d>10] = 0
    #ls_d[ls_d<-10] = -10
    img = nb.Nifti1Image(ls_d,affine=ls.affine,header=ls.header)
    img.to_filename(join(out_dir,sub_id,files+'_reg.nii.gz'))
    
    rows.append([join(out_dir,sub_id,files+'_reg.nii.gz')])

/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


In [7]:
files = "cortex_ls_r"

rows_2=[]
for sub_id in sub_ids:
    ls = nb.load(join(ls_dir,sub_id,files+".nii.gz"))
    ls_d = ls.get_data()
    ls_d[ls_d>10] = 0
    #ls_d[ls_d<-10] = -10
    img = nb.Nifti1Image(ls_d,affine=ls.affine,header=ls.header)
    img.to_filename(join("/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_r/output/",sub_id+"_"+files+'_reg.nii.gz'))
    
    if os.path.exists(join(out_dir,sub_id,sub_id+"_"+files+'_reg.nii.gz')):
        os.remove(join(out_dir,sub_id,sub_id+"_"+files+'_reg.nii.gz'))
    
    rows_2.append([join("/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_r/output/",sub_id+"_"+files+'_reg.nii.gz')])

/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  


In [8]:
ls_out_dir = "/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_r/output/"

with open(join(ls_out_dir,'ls_dirs.csv'), 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(rows_2)###########look here

writeFile.close()

In [ ]:
subprocess.call(["",""])

# Coreg Subcortex

In [7]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
ls_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/subcortex/output/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'

all_dirs = glob(join(source_dir,'*'))
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [8]:
files = "subcortex"

rows=[]
for sub_id in sub_ids:
    subprocess.call(["fslmaths",
                 join(ls_dir,sub_id,files+".nii.gz"),
                 "-dilM",
                 join(ls_dir,sub_id,files+"_reg.nii.gz")])
    subprocess.call(["fslmaths",
                 join(ls_dir,sub_id,files+"_reg.nii.gz"),
                 "-dilM",
                 join(ls_dir,sub_id,files+"_reg.nii.gz")])
    subprocess.call(["fslmaths",
                 join(ls_dir,sub_id,files+"_reg.nii.gz"),
                 "-dilM",
                 join(ls_dir,sub_id,files+"_reg.nii.gz")])
    if os.path.exists(join(out_dir,sub_id,files+'_R1_reg.nii.gz')):
        os.remove(join(out_dir,sub_id,files+'_R1_reg.nii.gz'))
        
    ls_d = nb.load(join(ls_dir,sub_id,files+"_reg.nii.gz")).get_data()
    r1 = nb.load(join(source_dir, sub_id, sub_id+"_R1.nii.gz"))
    r1_d = r1.get_data()
    r1_d[ls_d==0] = 0
    img = nb.Nifti1Image(r1_d,affine=r1.affine,header=r1.header)
    img.to_filename(join("/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/subcortex/output/",sub_id+"_"+files+'_R1_reg.nii.gz'))
    
    rows.append([join("/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/subcortex/output/",sub_id+"_"+files+'_R1_reg.nii.gz')])

In [9]:
ls_out_dir = "/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/subcortex/output/"

with open(join(ls_out_dir,'ls_dirs.csv'), 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(rows)###########look here

writeFile.close()

# Mapping data

In [8]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
data_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_data_dir = 'cortex_r_MD_R2_cap'  #goes in join(data_dir, sub_id, extra_data_dir)
mask_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/output/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_out_dir = 'mapped_reg_r_MD_R2_cap'

all_dirs = glob(source_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [2]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
data_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA'
extra_data_dir = 'cortex_MD_R2_cap'  #goes in join(data_dir, sub_id, extra_data_dir)
mask_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/output/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA'
extra_out_dir = 'mapped_reg_l_MD_R2_cap'

all_dirs = glob(source_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [3]:
import shutil
for ii in range(len(sub_ids)):

    sub_id = sub_ids[ii]
    in_dir = join(data_dir,sub_id, extra_data_dir)       
    #GM = nb.load(join(mask_dir,sub_id,'cortex_l.nii.gz'))

    all_data = glob(join(in_dir,sub_id+'*.nii.gz'))
    data = [os.path.basename(x).strip(".nii.gz") for x in all_data]
    
    for d in range(len(all_data)):
        output_dir = join(out_dir,sub_id,extra_out_dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print('Created your directory: {}'.format(output_dir))
            
        output_generic = join(output_dir,data[d]+"_generic.nii.gz")
        subprocess.call(["antsApplyTransforms", "-d", "3", 
                         "-i", all_data[d], 
                         "-r", join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), #"-r", join(reg_dir,"coreg_subcortex_template0warp.nii.gz"), 
                         "-o", join(output_dir,data[d]+"_generic_and_warp.nii.gz"), #output_generic, 
                         "-n", "NearestNeighbor", 
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"0GenericAffine.mat"),
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"1Warp.nii.gz")])
                         "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"0GenericAffine.mat"),
                         "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"1Warp.nii.gz")])
#                          "-t", join(reg_dir,"coreg_demons_full_"+sub_id+"_cortex_ls_l_reg"+str(ii)+"0GenericAffine.mat"),
#                          "-t", join(reg_dir,"coreg_demons_full_"+sub_id+"_cortex_ls_l_reg"+str(ii)+"1Warp.nii.gz")])


Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1003/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1004/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1005/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1006/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1007/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1009/mapped_reg_l_MD_R2_cap
Created 

Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3014/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3017/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3018/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3019/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3020/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3021/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3023/mapped_reg_l_MD_R2_cap
Created 

In [4]:
part_to_analyse = sub_ids

sum_data = np.zeros([176,200,160,len(part_to_analyse),6])
avg_ls = nb.load(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"))#avg_ls = nb.load(join(reg_dir,"coreg_subcortex_template0.nii.gz"))
output_dir = join(out_dir,"all",extra_out_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print('Created your directory: {}'.format(output_dir))

for ii in range(len(part_to_analyse)):
    
    sub_id = sub_ids[ii]
    in_dir = join(out_dir,sub_id,extra_out_dir)
    
    all_data = glob(join(in_dir,'*generic_and_warp.nii.gz'))
    all_data.sort()
    data = [os.path.basename(x).strip(".nii.gz").strip(sub_id) for x in all_data]
    print(all_data)
    
    
    for d in range(len(all_data)):
        #intensity prop not needed for subcortex
#         metric = nb.load(all_data[d])
        metric = nighres.intensity.intensity_propagation(image = all_data[d], 
                                                        mask=None, 
                                                        combine='mean', 
                                                        distance_mm=1.0, 
                                                        target='zero', 
                                                        scaling=1.0,
                                                        save_data=False)
        sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + metric['result'].get_data()
        #sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + nb.load(all_data[d]).get_data()
        

#also not needed for subcortex
mesh = nighres.surface.levelset_to_mesh(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), 
                                        connectivity='18/6', 
                                        level=0.0, 
                                        inclusive=True, 
                                        save_data=True, 
                                        overwrite=False, 
                                        output_dir=reg_dir, 
                                        file_name="coreg_demonstemplate0_final")

for d in range(len(all_data)):
    median = nb.Nifti1Image(np.median(sum_data[:,:,:,:,d],axis=3),affine=avg_ls.affine,header=avg_ls.header) #nb.Nifti1Image(sum_data[:,:,:,d]/(ii+1),affine=avg_ls.affine,header=avg_ls.header)
    median.to_filename(join(output_dir,'avg_'+data[d]+'.nii.gz'))
    #also not needed for subcortex
    nighres.surface.surface_mesh_mapping(intensity_image = median, 
                                         surface_mesh = mesh['result'], 
                                         inflated_mesh=None, 
                                         mapping_method='closest_point', 
                                         save_data=True, 
                                         overwrite=True, 
                                         output_dir=join(output_dir,"mapped"), 
                                         file_name=data[d])

Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/all/mapped_reg_l_MD_R2_cap
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap/sub1001_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap/sub1001_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap/sub1001_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap/sub1001_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_R2_cap/sub1001_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1001/mapped_reg_l_MD_

/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap/sub1002_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap/sub1002_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap/sub1002_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap/sub1002_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_cap/sub1002_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1002/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_cap/sub1012_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_cap/sub1012_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_cap/sub1012_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_cap/sub1012_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_cap/sub1012_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1012/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_cap/sub1025_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_cap/sub1025_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_cap/sub1025_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_cap/sub1025_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_cap/sub1025_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub1025/mapped_reg_l_MD_R2_ca

...


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_cap/sub3012_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_cap/sub3012_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_cap/sub3012_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_cap/sub3012_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_cap/sub3012_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3012/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap/sub3022_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap/sub3022_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap/sub3022_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap/sub3022_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_cap/sub3022_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3022/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_cap/sub3030_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_cap/sub3030_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_cap/sub3030_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_cap/sub3030_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_cap/sub3030_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3030/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_cap/sub3038_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_cap/sub3038_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_cap/sub3038_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_cap/sub3038_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_cap/sub3038_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3038/mapped_reg_l_MD_R2_ca


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_cap/sub3048_PC_1_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_cap/sub3048_PC_2_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_cap/sub3048_PC_3_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_cap/sub3048_PC_4_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_cap/sub3048_PC_5_cortex_MD_R2_cap_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Dim_Reduction/PCA/sub3048/mapped_reg_l_MD_R2_ca

In [5]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
data_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_data_dir = 'cortex_r_MD_R2_cap'  #goes in join(data_dir, sub_id, extra_data_dir)
mask_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_r/output/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_out_dir = 'mapped_reg_r_MD_R2_cap'

all_dirs = glob(source_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [6]:
import shutil
for ii in range(len(sub_ids)):

    sub_id = sub_ids[ii]
    in_dir = join(data_dir,sub_id, extra_data_dir)       
    #GM = nb.load(join(mask_dir,sub_id,'cortex_l.nii.gz'))

    all_data = glob(join(in_dir,sub_id+'*.nii.gz'))
    data = [os.path.basename(x).strip(".nii.gz") for x in all_data]
    
    for d in range(len(all_data)):
        output_dir = join(out_dir,sub_id,extra_out_dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print('Created your directory: {}'.format(output_dir))
            
        output_generic = join(output_dir,data[d]+"_generic.nii.gz")
        subprocess.call(["antsApplyTransforms", "-d", "3", 
                         "-i", all_data[d], 
                         "-r", join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), #"-r", join(reg_dir,"coreg_subcortex_template0warp.nii.gz"), 
                         "-o", join(output_dir,data[d]+"_generic_and_warp.nii.gz"), #output_generic, 
                         "-n", "NearestNeighbor", 
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"0GenericAffine.mat"),
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"1Warp.nii.gz")])
#                          "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"0GenericAffine.mat"),
#                          "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"1Warp.nii.gz")])
                         "-t", join(reg_dir,"coreg_demons_full_"+sub_id+"_cortex_ls_r_reg"+str(ii)+"0GenericAffine.mat"),
                         "-t", join(reg_dir,"coreg_demons_full_"+sub_id+"_cortex_ls_r_reg"+str(ii)+"1Warp.nii.gz")])


In [7]:
part_to_analyse = sub_ids

sum_data = np.zeros([176,200,160,len(part_to_analyse),6])
avg_ls = nb.load(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"))#avg_ls = nb.load(join(reg_dir,"coreg_subcortex_template0.nii.gz"))
output_dir = join(out_dir,"all",extra_out_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print('Created your directory: {}'.format(output_dir))

for ii in range(len(part_to_analyse)):
    
    sub_id = sub_ids[ii]
    in_dir = join(out_dir,sub_id,extra_out_dir)
    
    all_data = glob(join(in_dir,'*generic_and_warp.nii.gz'))
    all_data.sort()
    data = [os.path.basename(x).strip(".nii.gz").strip(sub_id) for x in all_data]
    print(all_data)
    
    
    for d in range(len(all_data)):
        #intensity prop not needed for subcortex
#         metric = nb.load(all_data[d])
        metric = nighres.intensity.intensity_propagation(image = all_data[d], 
                                                        mask=None, 
                                                        combine='mean', 
                                                        distance_mm=1.0, 
                                                        target='zero', 
                                                        scaling=1.0,
                                                        save_data=False)
        sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + metric['result'].get_data()
        #sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + nb.load(all_data[d]).get_data()
        

#also not needed for subcortex
mesh = nighres.surface.levelset_to_mesh(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), 
                                        connectivity='18/6', 
                                        level=0.0, 
                                        inclusive=True, 
                                        save_data=True, 
                                        overwrite=False, 
                                        output_dir=reg_dir, 
                                        file_name="coreg_demonstemplate0_final")

for d in range(len(all_data)):
    median = nb.Nifti1Image(np.median(sum_data[:,:,:,:,d],axis=3),affine=avg_ls.affine,header=avg_ls.header) #nb.Nifti1Image(sum_data[:,:,:,d]/(ii+1),affine=avg_ls.affine,header=avg_ls.header)
    median.to_filename(join(output_dir,'avg_'+data[d]+'.nii.gz'))
    #also not needed for subcortex
    nighres.surface.surface_mesh_mapping(intensity_image = median, 
                                         surface_mesh = mesh['result'], 
                                         inflated_mesh=None, 
                                         mapping_method='closest_point', 
                                         save_data=True, 
                                         overwrite=True, 
                                         output_dir=join(output_dir,"mapped"), 
                                         file_name=data[d])

['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/mapped_reg_r_MD_R2_cap/sub1001_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation


/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/mapped_reg_r_MD_R2_cap/sub1002_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1012/mapped_reg_r_MD_R2_cap/sub1012_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1025/mapped_reg_r_MD_R2_cap/sub1025_R2s_OLS_generic_and_warp.nii.gz

...


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3012/mapped_reg_r_MD_R2_cap/sub3012_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/mapped_reg_r_MD_R2_cap/sub3022_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3030/mapped_reg_r_MD_R2_cap/sub3030_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3038/mapped_reg_r_MD_R2_cap/sub3038_R2s_OLS_generic_and_warp.nii.gz


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3048/mapped_reg_r_MD_R2_cap/sub3048_R2s_OLS_generic_and_warp.nii.gz

In [23]:
import nighres as nr; 
import nibabel as nb;
yy = nb.load('/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/xxx.nii.gz'); 
yyy = yy.get_data(); 
import numpy; 
yy[yy==0] = numpy.amax(yy);
average = nb.Nifti1Image(yyy,affine=yy.affine,header=yy.header);
average.to_filename('/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/x2.nii.gz');  
nr.surface.levelset_to_mesh('/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/x2.nii.gz',save_data=True,output_dir='/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/',overwrite=True)


TypeError: 'Nifti1Image' object does not support item assignment

# Standard Dev mapping

In [3]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
data_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_data_dir = 'cortex_l'  #goes in join(data_dir, sub_id, extra_data_dir)
mask_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
reg_dir = '/data/neuralabc/carfra/QuantMetComp/scripts/ANTs/cortex_l/output/'
out_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
extra_out_dir = 'cortex_sd'

all_dirs = glob(source_dir+'*')
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [4]:
import shutil
for ii in range(len(sub_ids)):

    sub_id = sub_ids[ii]
    in_dir = join(data_dir,sub_id, extra_data_dir)       
    #GM = nb.load(join(mask_dir,sub_id,'cortex_l.nii.gz'))

    all_data = glob(join(in_dir,sub_id+'*.nii.gz'))
    data = [os.path.basename(x).strip(".nii.gz") for x in all_data]
    
    for d in range(len(all_data)):
        output_dir = join(out_dir,sub_id,extra_out_dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print('Created your directory: {}'.format(output_dir))
            
        output_generic = join(output_dir,data[d]+"_generic.nii.gz")
        subprocess.call(["antsApplyTransforms", "-d", "3", 
                         "-i", all_data[d], 
                         "-r", join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), #"-r", join(reg_dir,"coreg_subcortex_template0warp.nii.gz"), 
                         "-o", join(output_dir,data[d]+"_generic_and_warp.nii.gz"), #output_generic, 
                         "-n", "NearestNeighbor", 
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"0GenericAffine.mat"),
                         #"-t", join(reg_dir,"coreg_subcortex_"+sub_id+"_subcortex_R1_reg"+str(ii)+"1Warp.nii.gz")])
                         "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"0GenericAffine.mat"),
                         "-t", join(reg_dir,"coreg_demons"+sub_id+"_cortex_ls_l_reg"+str(ii)+"1Warp.nii.gz")])

Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1003/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1004/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1005/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1006/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1007/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1009/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetC

Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3019/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3020/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3021/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3023/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3024/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3025/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3026/cortex_sd
Created your directory: /data/neuralabc/carfra/QuantMetC

In [5]:
part_to_analyse = sub_ids

sum_data = np.zeros([176,200,160,len(part_to_analyse),6])
avg_ls = nb.load(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"))#avg_ls = nb.load(join(reg_dir,"coreg_subcortex_template0.nii.gz"))
output_dir = join(out_dir,"all",extra_out_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print('Created your directory: {}'.format(output_dir))

for ii in range(len(part_to_analyse)):
    
    sub_id = sub_ids[ii]
    in_dir = join(out_dir,sub_id,extra_out_dir)
    
    all_data = glob(join(in_dir,'*generic_and_warp.nii.gz'))
    all_data.sort()
    data = [os.path.basename(x).strip(".nii.gz").strip(sub_id) for x in all_data]
    print(all_data)
    
    
    for d in range(len(all_data)):
        #intensity prop not needed for subcortex
        metric = nighres.intensity.intensity_propagation(image = all_data[d], 
                                                        mask=None, 
                                                        combine='mean', 
                                                        distance_mm=2.0, 
                                                        target='zero', 
                                                        scaling=1.0,
                                                        save_data=False)
        sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + metric['result'].get_data()
        #sum_data[:,:,:,ii,d] = sum_data[:,:,:,ii,d] + nb.load(all_data[d]).get_data()
        

#also not needed for subcortex
mesh = nighres.surface.levelset_to_mesh(join(reg_dir,"coreg_demonstemplate0_final.nii.gz"), 
                                        connectivity='18/6', 
                                        level=0.0, 
                                        inclusive=True, 
                                        save_data=True, 
                                        overwrite=False, 
                                        output_dir=reg_dir, 
                                        file_name="coreg_demonstemplate0_final")

for d in range(len(all_data)):
    std = nb.Nifti1Image(np.std(sum_data[:,:,:,:,d],axis=3),affine=avg_ls.affine,header=avg_ls.header) #nb.Nifti1Image(sum_data[:,:,:,d]/(ii+1),affine=avg_ls.affine,header=avg_ls.header)
    std.to_filename(join(output_dir,'avg_'+data[d]+'.nii.gz'))
    #also not needed for subcortex
    nighres.surface.surface_mesh_mapping(intensity_image = std, 
                                         surface_mesh = mesh['result'], 
                                         inflated_mesh=None, 
                                         mapping_method='closest_point', 
                                         save_data=True, 
                                         overwrite=True, 
                                         output_dir=join(output_dir,"mapped"), 
                                         file_name=data[d])

Created your directory: /data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/all/cortex_sd
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1001/cortex_sd/sub1001_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation


/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1002/cortex_sd/sub1002_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1016/cortex_sd/sub1016_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub1027/cortex_sd/sub1027_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten

...


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3011/cortex_sd/sub3011_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3022/cortex_sd/sub3022_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3031/cortex_sd/sub3031_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3040/cortex_sd/sub3040_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten


Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation

Intensity Propagation
['/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_FA_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_MD_re_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_MTsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_PDsat_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_R1_generic_and_warp.nii.gz', '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/sub3051/cortex_sd/sub3051_R2s_OLS_generic_and_warp.nii.gz']

Intensity Propagation

Intensity Propagation

Intensity Propagation

Inten

# masking metrics

In [6]:
source_dir = '/data/neuralabc/source/MPI_CBS/MPM_DTI/source/'
ls_dir = '/data/neuralabc/carfra/QuantMetComp/source/masks_created/'
# reg_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/Coreg/l_coreg_surf_all/'
#out_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
output_dir = '/data/neuralabc/carfra/QuantMetComp/processing/MPM/MPM_correlations/metrics/'
mask_n = 'cortex_r_MD_R2_cap'

all_dirs = glob(join(source_dir,'*'))
sub_ids = [os.path.basename(x) for x in all_dirs]
name = ['FA','MD','MTsat','PDsat','R1','$R2^*$']  #hard-coded, but I know what they are!

In [7]:
for sub_id in sub_ids:
    all_met = glob(join(source_dir,sub_id,'*.nii.gz'))
    for met in all_met:
        met_name = os.path.basename(met)
        
        met_f = nb.load(met)
        met_d = met_f.get_data()
        
        mask = nb.load(join(ls_dir, sub_id, mask_n + '.nii.gz'))
        mask_d = mask.get_data()
        
        met_masked = met_d*mask_d
        
        if not os.path.exists(join(output_dir,sub_id,mask_n)):
            os.makedirs(join(output_dir,sub_id,mask_n))
        average = nb.Nifti1Image(met_masked,affine=met_f.affine,header=met_f.header)
        average.to_filename(join(output_dir,sub_id,mask_n,met_name))
        
        if os.path.isfile(join(out_dir,sub_id,met_name)):
            os.remove(join(out_dir,sub_id,met_name))
        

/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  import sys
/opt/quarantine/anaconda3/2019.10/install/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  # Remove the CWD from sys.path while we load stuff.


In [1]:
nighres.surface.levelset_to_mesh(levelset_image = '/data/neuralabc/carfra/QuantMetComp/scripts/twolevel_ants_dbm/output/secondlevel/Border_removed.nii.gz', 
                                 connectivity='18/6', 
                                 level=0.0, 
                                 inclusive=True, 
                                 save_data=True, 
                                 overwrite=True, 
                                 output_dir='/data/neuralabc/carfra/QuantMetComp/scripts/twolevel_ants_dbm/output/secondlevel/', 
                                 file_name='mesh_border_removed')

NameError: name 'nighres' is not defined

In [ ]:
aaa.shape